# Exercises

Learning a language requires _using_ it. This is why I can no longer speak French. 

This page gives gives a series of exercises for basic analyses of data in the Datajoint pipeline. 

If you have not yet reconfigured your Datajoint client to access the latest version of the pipeline, you can find details [on the kavli wiki here](https://www.ntnu.no/wiki/display/kavli/DataJoint%3A+Electrophysiology+Pipeline). You will require your NTNU username, database password, and the access and secret keys to the object storage. 

In [ ]:
import os
import scipy.ndimage
import datetime
import numpy as np
import datajoint as dj
import matplotlib.pyplot as plt

SECRET_KEY = ""
ACCESS_KEY = ""
USER = ''
PASS = ''

dj.config['database.host'] = 'datajoint.it.ntnu.no'
dj.config['database.user'] = USER
dj.config['database.password'] = PASS
dj.config["enable_python_native_blobs"] = True
dj.config["cache"] = "C:/temp/djcache"
dj.config["stores"] = {   'ephys_store': {   'access_key': ACCESS_KEY,
                                     'bucket': 'ephys-store-computed',
                                     'endpoint': 's3.stack.it.ntnu.no:443',
                                     'secure': True,
                                     'location': '',
                                     'protocol': 's3',
                                     'secret_key': SECRET_KEY},
                  'ephys_store_manual': {   'access_key': ACCESS_KEY,
                                            'bucket': 'ephys-store-manual',
                                            'endpoint': 's3.stack.it.ntnu.no:443',
                                            'secure': True,
                                            'location': '',
                                            'protocol': 's3',
                                            'secret_key': SECRET_KEY}}
dj.config['custom'] = {
        'database.prefix': 'group_shared_',
        'mlims.database': 'prod_mlims_data',
        'flask.database': 'group_shared_flask',
        'drive_config': {
          'local': 'C:/',
          'network': 'N:/'}}
# animal = dj.create_virtual_module('animal', 'prod_mlims_data')
# reference = dj.create_virtual_module('reference', 'group_shared_reference')
# acquisition = dj.create_virtual_module('acquisition', 'group_shared_acquisition')
# tracking = dj.create_virtual_module('tracking', 'group_shared_tracking')
# behavior = dj.create_virtual_module('behavior', 'group_shared_behavior')
# ephys = dj.create_virtual_module('ephys', 'group_shared_ephys')
analysis = dj.create_virtual_module('analysis', 'group_shared_analysis')
analysis_param = dj.create_virtual_module('analysis_param', 'group_shared_analysis_param')

key = {'animal_id': 'f5e35afe2bf2a9f8',
 'datasource_id': 0,
 'session_time': datetime.datetime(2019, 11, 11, 12, 45, 59),
 'unit': 167,
 'task_type': 'OpenField',
 'task_start': '21.00',
 'task_spike_tracking_hash': '083ff30e220fbe26815e820e54451af6',
 'occu_params_name': 'default',
 'smoothing_params_name': 'default',
 'analysis_package': 'python',
 'cell_selection_params_name': 'default',
 'field_detect_params_name': 'default',
 'score_params_name': 'default'}

## Exercise 1: Path maps

In the first exercise, you will plot a path and spike map

Goal: given a `key` to the Datajoint table `analysis.TaskSpikesTracking`, create and display a path Map for that cell in that task. 

This one is fairly simple: due to the convenience of Datajoint, you need only fetch four arrays from the database, and plot them directly using `matplotlib`. Don't forget to set the correct aspect ratio. Further, because this is about using Python correctly, not Datajoint in particular, the below code will do the database hevay lifting for you

In [ ]:
pos_t, pos_x, pos_y = (analysis.TaskTracking & key).fetch1("task_timestamps", "x_pos", "y_pos")
spk_t, spk_x, spk_y = (analysis.TaskSpikesTracking & key).fetch1("spike_times", "x_pos", "y_pos")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(pos_x, pos_y, "C1-", linewidth=0.3)
ax.scatter(spk_x, spk_y, s=4)
ax.set_aspect("equal")
ax.set_xlabel("Position [cm]")
ax.set_ylabel("Position [cm]")
ax.set_title("Path map")

## Exercise 2: Coverage

It's useful to know how much of the arena the animal has visited. This requires some manipulation of the data you have acquired so far. IWhat assumptions are you making?

**Hint**: You will find the [NumPy Documentation](https://docs.scipy.org/doc/numpy) very useful, and the [2D histogram](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram2d.html) function in particular, and it's hard to get anything done with arrays without [meshgrid](https://docs.scipy.org/doc/numpy/reference/generated/numpy.meshgrid.html)

In [ ]:
num_bins = 30
time_map, xedge, yedge = np.histogram2d(pos_x, pos_y, bins=num_bins)
np.count_nonzero(time_map) / time_map.size

In [ ]:
distance = np.ones(time_map.size)
bin_width = abs(xedge[1] - xedge[0])/2
X, Y = np.meshgrid(xedge[:-1]-bin_width, yedge[:-1]-bin_width)
distance = np.sqrt(X**2 + Y**2)
time_map[distance>75] = np.nan
np.ma.sum(time_map>0) / (time_map.size - np.sum(np.isnan(time_map)))

## Exercise 3: Rate maps

A ratemap takes in a list of animal positions, and a list of spike positions, and computes the rate of firing of that unit at each location in an arena. 

Based on these data, create:
* An occupancy map: a binned map of the time the animal spent in each location
* A spike map: a binned map of the number of unit spikes in each location
* A rate map: the rate at which the unit fires in each location





In [ ]:
seconds_per_frame = np.mean(np.diff(pos_t))
time_map_frames, xedge, yedge = np.histogram2d(pos_x, pos_y, bins=num_bins)
time_map = time_map_frames * frames_per_second
spike_map, xedge, yedge = np.histogram2d(spk_x, spk_y, bins=num_bins)
rate_map = spike_map / time_map


fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12, 16))
ax1.imshow(time_map)
ax1.set_title("Time map")
ax2.imshow(spike_map)
ax2.set_title("Spike map")
ax3.imshow(rate_map)
ax3.set_title("Rate map")


# Exercise 4: Smoothing

Coarse discretisation like this is highly prone to distortionary boundary effects, which can be ameliorated by smoothing. Conventionally, by smoothing, we mean that each pixel is made more like its neighbours. The fascinating detail (and a significant part of the engineering-physics-maths sub-field of signal processing) is how neighbours are weighted in this calculation

This is where analysis starts to become extremely complicated by the distressing existence of boundaries. 

How we we handle unexpected values?

The simplest solution for convolution in Python is to use [`scipy.ndimage.filters.convolve`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.ndimage.filters.convolve.html)

This requires a `weights` keyword that determines how we weight the value of each neighbour.

In [ ]:
rate_map_smooth = scipy.ndimage.filters.convolve(rate_map, weights=np.ones((3,3)))

In [ ]:
plt.imshow(rate_map_smooth)

### Boundary handling

As you can see, the borders of the image have shrunk, and the invalid numbers have spread - because if you take an average of a number, and not-a-number, you get not-a-number. 

We can be a bit more clever about this: we can replace the NaN values with something more meaningful. The obvious choice here is zero - locations that the animal never visited _cannot_ have a meaningful spike rate

In [ ]:
rate_map_finite = rate_map.copy()
rate_map_finite[np.isnan(rate_map)] = 0
rate_map_finite_smooth = scipy.ndimage.filters.convolve(rate_map_finite, weights=np.ones((3,3)))
plt.imshow(rate_map_finite_smooth)

### More boundaries

The effect of convolution is to "smear out" the data, but that still has several problems:
1. We're using a "square-edged" smear - the choice of weighting leaves something to be desired
2. We now have non-zero rates in locations that the animal physically could not visit _because they are outside the box_

Solving point 2 is easy: we can re-apply an outside limit using the `distance` array we calculated earlier

In [ ]:
rate_map_finite_smooth[distance>75] = 0
plt.imshow(rate_map_finite_smooth)

Solving point 1 means using a different `weights` keyword from earlier. A common choice is a Gaussian blur - meaning that values at a greater difference are weighted exponentially lower, according to the Gaussian distribution given by _sigma_

We could write our own `weights` array based on a Gaussian... or we could just use the conveniently provided [scipy.ndimage.gaussian_filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html)

In [ ]:
rate_map_finite_smooth_gaussian = scipy.ndimage.gaussian_filter(rate_map_finite, sigma=1.5)
rate_map_finite_smooth_gaussian[distance>75] = 0
plt.imshow(rate_map_finite_smooth_gaussian)

We can also format this better

In [ ]:
fig, ax = plt.subplots(figsize=(6,8))

image = ax.imshow(rate_map_finite_smooth_gaussian, cmap="gist_earth")
ax.set_axis_off()
ax.set_aspect("equal")
cax = fig.add_axes([0.95, 0.2125, 0.05, 0.585])
cbar = fig.colorbar(image, cax=cax, orientation="vertical")
ax.set_title("Rate Map")
plt.show()